In [1]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("../csv/AAPL.csv", index_col="Date")
df.head(2)

In [ ]:
#결측치 조회 (NaN, 음의 무한대, 양의 무한대)
df[df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [10]:
# 결측치를 제외한 데이터를 출력
# df.dropna(axis=0)
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
## 종가만 가지고 테스트
## 시간과 종가만 있는 데이터프레임 
price_df = df.loc[:, ['Adj Close']]
price_df

In [ ]:
price_df.plot()

In [ ]:
## 일별 수익율 파생변수 생성 ( 오늘날짜의 종가 - 어제 날짜의 종가) / 어제 날짜의 종가
price_df["daily_rtn"] = price_df["Adj Close"].pct_change()
#price_df["Adj Close"].diff()/price_df["Adj Close"].shift()
# price_df.head()

In [ ]:
## 일별 수익율의 누적곱 파생변수 생성
price_df["st_rtn"] = (1 + price_df["daily_rtn"]).cumprod()
price_df.head()

In [ ]:
price_df["st_rtn"].plot()

In [ ]:
first_date = price_df.index[0]
last_date = price_df.index[-1]
print(first_date, last_date)

In [ ]:
print(price_df.loc[last_date, 'Adj Close'] / price_df.loc[first_date, 'Adj Close'])

In [ ]:
print(price_df.loc[last_date, 'st_rtn'])

In [ ]:
# backtesting 
# cummax() - 종가에서 최고가 저장
# cummin() - 최대 하락 계산
historical_max = price_df["Adj Close"].cummax()
daily_drawdown = price_df["Adj Close"] / historical_max - 1.0
historical_min = daily_drawdown.cummin()
historical_min.plot()

In [27]:
## CAGR : 일별 date 사용 -> 
# 전체 연도를 계산하는 경우 전체 데이터의 기간을 252 영업일을 기준으로 
# 나눈 역수를 제곱한 후 -1
## MDD : 최대 낙폭
## VOL : 연 주가 수익률 표준 편차 = 일별 주가 수익률 * 252일 제곱근
## sharpe : 총 위험 1단위당 초과 수익률
CAGR = price_df.loc[last_date, 'st_rtn'] ** (252/len(price_df))-1
sharpe = np.mean(price_df["daily_rtn"])/np.std(price_df["daily_rtn"])*np.sqrt(252)
VOL = np.std(price_df["daily_rtn"]) * np.sqrt(252)
MDD = historical_min.min()

In [28]:
print(VOL)
print('CAGR : ', round(CAGR*100, 2), '%')
print('Sharpe : ', round(sharpe, 2))
print("VOL : ", round(VOL*100, 2), '%')
print("MDD : ", round(-1*MDD*100, 2), '%')

0.4570483127920232
CAGR :  17.4 %
Sharpe :  0.58
VOL :  45.7 %
MDD :  81.8 %


In [ ]:
import buy_and_hold as b_h
import imp
imp.reload(b_h)

In [30]:
df = pd.read_csv("../csv/AMZN.csv", index_col="Date")

In [31]:
## 클래스 생성
bh_df = b_h.buyandhold(df, 'Adj Close')

In [ ]:
## 결측치 제거
bh_df.drop_na()

In [ ]:
## 파생변수 생성 함수 add_column
bh_df.add_column()

In [35]:
result = bh_df.testing()

In [36]:
CAGR = result[0]
sharpe = result[1]
VOL = result[2]
MDD = result[3]

print(VOL)
print('CAGR : ', round(CAGR*100, 2), '%')
print('Sharpe : ', round(sharpe, 2))
print("VOL : ", round(VOL*100, 2), '%')
print("MDD : ", round(-1*MDD*100, 2), '%')

0.6008984712355606
CAGR :  36.58 %
Sharpe :  0.81
VOL :  60.09 %
MDD :  94.4 %


In [ ]:
!pip install finance-datareader

In [ ]:
!pip install bs4

In [39]:
import FinanceDataReader as fdr

In [43]:
ko_df = fdr.DataReader('KS11', '2020-01-01', '2022-09-10')
ko_df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2020-01-02,2175.17,2201.21,2202.32,2171.84,494680000.0,-0.0102
2020-01-03,2176.46,2192.58,2203.38,2165.39,631560000.0,0.0006
2020-01-06,2155.07,2154.97,2164.42,2149.95,592670000.0,-0.0098
2020-01-07,2175.54,2166.60,2181.62,2164.27,568240000.0,0.0095
2020-01-08,2151.31,2156.27,2162.32,2137.72,913830000.0,-0.0111
...,...,...,...,...,...,...
2022-09-02,2409.41,2427.70,2432.37,2402.67,391600000.0,-0.0026
2022-09-05,2403.68,2410.07,2424.77,2392.63,354740000.0,-0.0024
2022-09-06,2410.02,2409.72,2422.53,2399.26,294220000.0,0.0026


In [44]:
bh_df_2 = b_h.buyandhold(ko_df, 'Close')

In [ ]:
bh_df_2.drop_na()

In [ ]:
bh_df_2.add_column()

In [48]:
result = bh_df_2.testing()

In [49]:
CAGR = result[0]
sharpe = result[1]
VOL = result[2]
MDD = result[3]

print(VOL)
print('CAGR : ', round(CAGR*100, 2), '%')
print('Sharpe : ', round(sharpe, 2))
print("VOL : ", round(VOL*100, 2), '%')
print("MDD : ", round(-1*MDD*100, 2), '%')

0.21954966989801789
CAGR :  3.53 %
Sharpe :  0.27
VOL :  21.95 %
MDD :  35.71 %
